In [62]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.regularizers import l2

In [63]:
CONFIG = {
    'data_path': '../dataset/',
    'image_size': (160, 160),# downlgaded from 224,224 to 160,160
    'batch_size': 64,  # Increased batch size
    'epochs': 30,      # Reduced epochs
    'initial_learning_rate': 0.001,  # Lower initial learning rate
    'num_classes': 10,
    'validation_split': 0.2,
    'weight_decay': 0.0005,
    'dropout_rate': 0.3,
    'early_stopping_patience': 8,  # Reduced patience
    'min_epochs': 10   # Reduced minimum epochs
}

In [64]:
def display_sample_images(data_path, num_samples=2):
    """
    Display sample images from each category
    """
    categories = os.listdir(data_path)
    plt.figure(figsize=(15, 10))
    
    for idx, category in enumerate(categories):
        category_path = os.path.join(data_path, category)
        images = os.listdir(category_path)[:num_samples]
        
        for i, image in enumerate(images):
            plt.subplot(len(categories), num_samples, idx * num_samples + i + 1)
            img = Image.open(os.path.join(category_path, image))
            plt.imshow(img)
            plt.title(f'{category} - Sample {i+1}')
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def dataset_statistics(data_path):
    """
    Display dataset statistics
    """
    categories = os.listdir(data_path)
    stats = {}
    
    for category in categories:
        category_path = os.path.join(data_path, category)
        num_images = len(os.listdir(category_path))
        stats[category] = num_images
    
    # Plot statistics
    plt.figure(figsize=(12, 6))
    plt.bar(stats.keys(), stats.values())
    plt.xticks(rotation=45)
    plt.title('Number of Images per Category')
    plt.xlabel('Category')
    plt.ylabel('Number of Images')
    plt.tight_layout()
    plt.show()
    
    return pd.DataFrame.from_dict(stats, orient='index', columns=['Count'])

In [65]:
def create_optimized_generators(config):
    """
    More efficient data generators
    """
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        validation_split=config['validation_split']
    )
    
    train_generator = train_datagen.flow_from_directory(
        config['data_path'],
        target_size=config['image_size'],
        batch_size=config['batch_size'],
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    
    val_generator = train_datagen.flow_from_directory(
        config['data_path'],
        target_size=config['image_size'],
        batch_size=config['batch_size'],
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    return train_generator, val_generator

In [66]:
def build_model(config):
    """
    Lighter model architecture for faster training
    """
    # Use a smaller EfficientNet model
    base_model = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(*config['image_size'], 3)
    )
    
    # Freeze most layers, only unfreeze the last few
    base_model.trainable = False
    for layer in base_model.layers[-10:]:  # Only unfreeze last 10 layers
        layer.trainable = True
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu', kernel_regularizer=l2(config['weight_decay'])),
        layers.Dropout(config['dropout_rate']),
        layers.Dense(config['num_classes'], activation='softmax', dtype='float32')
    ])
    
    return model

In [67]:
def calculate_class_weights(train_generator):
    """
    Calculate balanced class weights based on the dataset statistics
    """
    class_counts = train_generator.classes.shape[0]
    class_weights = {}
    
    # Using inverse class frequency with smoothing
    for class_idx in range(train_generator.num_classes):
        count = np.sum(train_generator.classes == class_idx)
        weight = (1 / count) * (class_counts / train_generator.num_classes)
        # Apply smoothing to prevent extreme weights
        weight = np.sqrt(weight)
        class_weights[class_idx] = weight
    
    return class_weights

In [68]:
def train_model(model, train_generator, val_generator, config):
    """
    Optimized training function
    """
    callbacks = [
        ModelCheckpoint(
            'best_model.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=config['early_stopping_patience'],
            restore_best_weights=True,
            mode='min',
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    # Enable mixed precision
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    
    history = model.fit(
        train_generator,
        epochs=config['epochs'],
        validation_data=val_generator,
        callbacks=callbacks
    )
    
    return history

In [69]:
def predict_document(model, image_path, config):
    """
    Make predictions on new images
    """
    # Load and preprocess image
    img = tf.keras.preprocessing.image.load_img(
        image_path,
        target_size=config['image_size']
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    
    # Make prediction
    prediction = model.predict(img_array)
    return prediction

In [70]:
def create_tf_dataset(generator, config):
    return tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32)
    ).prefetch(tf.data.AUTOTUNE)

In [ ]:

def main():
    try:
        print("Creating data generators...")
        train_generator, val_generator = create_optimized_generators(CONFIG)
        
        # Set mixed precision policy
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
        
        # Build model
        print("Building and compiling model...")
        model = build_model(CONFIG)
        
        # Explicitly compile the model with float32 metrics
        optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG['initial_learning_rate'])
        optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
        
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=[
                tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                tf.keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy', dtype='float32')
            ]
        )
        
        print("Model compiled successfully")
        print("Starting training...")
        history = train_model(model, train_generator, val_generator, CONFIG)
        
        return model, history
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None
# Clean up - remove any duplicate calls
if __name__ == "__main__":
    # Enable mixed precision training
    # tf.keras.mixed_precision.set_global_policy('mixed_float16')
    
    # Run main function
    model, history = main()

Creating data generators...
Found 2788 images belonging to 10 classes.
Found 694 images belonging to 10 classes.
Building and compiling model...
Model compiled successfully
Starting training...


c:\Users\Elgue\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1703 - loss: 3.2516 - top_2_accuracy: 0.3014